Tkinter is a Python library for creating graphical user interfaces (GUIs) that provides a simple and intuitive way to build desktop applications. With Tkinter, developers can easily create interactive interfaces by arranging and configuring a variety of pre-built widgets such as buttons, labels, and text boxes. Tkinter follows an event-driven programming model, where user interactions trigger events that can be handled by event handlers or callback functions, allowing for responsive and interactive applications. It is cross-platform, making it compatible with Windows, macOS, and Linux, and seamlessly integrates with Python, enabling developers to leverage Python's extensive ecosystem for building sophisticated GUI applications with ease.
The below code is for the GUI. The GAN Architecture is saved in the form of a h5 file. The file has already been attached. The GAN Architecture has already been trained. Please keep them in the same folder while running this code.

In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from matplotlib import pyplot as plt
import os
import time

def close_window():
    root.destroy()

def preprocess_image(image_path):
    #Loads image from path and preprocesses to make it model ready
    hr_image = tf.image.decode_image(tf.io.read_file(image_path))
    if hr_image.shape[-1] == 4:
        hr_image = hr_image[...,:-1]
    hr_size = (tf.convert_to_tensor(hr_image.shape[:-1]) // 4) * 4
    hr_image = tf.image.crop_to_bounding_box(hr_image, 0, 0, hr_size[0], hr_size[1])
    hr_image = tf.cast(hr_image, tf.float32)
    return tf.expand_dims(hr_image, 0)

def image_processing(image_path):
    print("Applying Super-Resolution using GAN model")
    gan_model = tf.keras.models.load_model("GAN_image_denoiser_1b-500ep_earlyStopped.h5")
    processed_image = preprocess_image(image_path)
    denoised_image = gan_model.predict(processed_image)
    denoised_image = denoised_image.astype(np.uint8)
    return denoised_image

def select_image():
    global image_path
    image_path = filedialog.askopenfilename()
    if image_path:
        messagebox.showinfo("Image Selected", f"You have selected the image from:\n{image_path}")
        process_button.config(state=tk.NORMAL)
        
def image_process(image_path):
    img = cv2.imread(image_path)
    dst = cv2.fastNlMeansDenoisingColored(img, None, 11, 10, 7, 21)
    kernel = np.array([[-0.5, -0.5, -0.5], [-0.5,  5.0, -0.5],[-0.5, -0.5, -0.5]])
    sharpened_image = cv2.filter2D(dst, -1, kernel)
    return sharpened_image

def process_image():
    global denoised_img
    process_button.config(state=tk.DISABLED)
    progress_var = tk.DoubleVar()
    progress_var.set(0)
    root.update_idletasks()
    denoised_img = image_process(image_path)
    denoised_img = cv2.cvtColor(denoised_img, cv2.COLOR_BGR2RGB)
    progress_var.set(100)
    root.update_idletasks()
    process_button.config(state=tk.NORMAL)
    display_denoised_image_in_new_page(denoised_img)

def display_denoised_image_in_new_page(denoised_img):
    new_window = tk.Toplevel(root)
    new_window.title("Denoised Image")
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    new_window.attributes("-fullscreen", True)
    background_img = Image.open("Background1.jpg")
    background_img = background_img.resize((screen_width, screen_height), Image.LANCZOS)
    background_img_tk = ImageTk.PhotoImage(background_img)
    background_label = tk.Label(new_window, image=background_img_tk)
    background_label.place(relx=0, rely=0, relwidth=1, relheight=1)
    background_label.image = background_img_tk
    denoised_pil_img = Image.fromarray(denoised_img)
    denoised_img_tk = ImageTk.PhotoImage(denoised_pil_img)
    denoised_label = tk.Label(new_window, image=denoised_img_tk)
    denoised_label.place(relx=0.5, rely=0.5, anchor="center")
    denoised_label.image = denoised_img_tk
    save_button = tk.Button(new_window, text="Save Denoised Image", command=save_denoised_image, font=("Palatino Linotype", 24, "bold"), bg="white", fg="black")
    save_button.pack(side=tk.BOTTOM, padx=20, pady=(0, 20))
    close_button = tk.Button(new_window, text="x", command=new_window.destroy, bg="red", fg="white", font=("Helvetica", 14))
    close_button.place(relx=0.97, rely=0.01, relwidth=0.03, relheight=0.03)


def save_denoised_image():
    global denoised_img
    if denoised_img is None:
        messagebox.showerror("Error", "No denoised image to save.")
        return
    save_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG files", "*.png"), ("JPEG files", "*.jpg"), ("All files", "*.*")])
    if save_path:
        try:
            cv2.imwrite(save_path + "_denoised.png", cv2.cvtColor(denoised_img, cv2.COLOR_BGR2RGB))
            messagebox.showinfo("Success", "Denoised image saved successfully.")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to save denoised image: {str(e)}")

root = tk.Tk()
root.title("Image Denoiser")
root.attributes("-fullscreen", True)

background_img = Image.open("Background.jpg")
background_img = background_img.resize((root.winfo_screenwidth(), root.winfo_screenheight()), Image.LANCZOS)
background_img = ImageTk.PhotoImage(background_img)

background_label = tk.Label(root, image=background_img)
background_label.place(relx=0, rely=0, relwidth=1, relheight=1)

feature_box_style = {
    "borderwidth": 2,
    "relief": tk.RAISED,
}

select_feature_frame = tk.Canvas(root, bg="#3181ed", **feature_box_style)
select_feature_frame.place(relx=0.175, rely=0.2, relwidth=0.3, relheight=0.3)

select_button = tk.Button(select_feature_frame, text="Select Image", command=select_image, font=("Palatino Linotype", 30))
select_button.place(relx=0.1, rely=0.1, relwidth=0.8, relheight=0.8)

process_feature_frame = tk.Canvas(root, bg="#0a59c3", **feature_box_style)
process_feature_frame.place(relx=0.55, rely=0.2, relwidth=0.3, relheight=0.3)

process_button = tk.Button(process_feature_frame, text="Process Image", command=process_image, state=tk.DISABLED, font=("Palatino Linotype", 30))
process_button.place(relx=0.1, rely=0.1, relwidth=0.8, relheight=0.8)

# Add the third feature box
flip_feature_frame = tk.Canvas(root, bg="#0c43ed", **feature_box_style)
flip_feature_frame.place(relx=0.3, rely=0.6, relwidth=0.4, relheight=0.3)

front_text = flip_feature_frame.create_text(150, 55, anchor="nw", text="Press to learn\n  How to use", font=("Palatino Linotype", 40), state="normal", fill="white")
back_text = flip_feature_frame.create_text(10, 10, anchor="nw", text="Steps to Denoise an image:\n\n1) Press the \"Select Image\" button and select the image that u want\n    to denoise.  (Process Image button unlocks after selection)\n\n2) Then press the \"Process Image\" button to denoise the image.\n\n3) A new window will open displaying the denoised image with\n    an option to save it, click it to save it in your computer.", font=("Palatino Linotype", 14), state="hidden", fill="white")

flip_feature_frame.flipped = False
flip_feature_frame.bind("<Button-1>", lambda event: flip_feature_box())

def flip_feature_box():
    # Toggle between front and back sides
    if flip_feature_frame.flipped:
        flip_feature_frame.itemconfig(front_text, state="normal")
        flip_feature_frame.itemconfig(back_text, state="hidden")
    else:
        flip_feature_frame.itemconfig(front_text, state="hidden")
        flip_feature_frame.itemconfig(back_text, state="normal")
    
    flip_feature_frame.flipped = not flip_feature_frame.flipped

close_button = tk.Button(root, text="x", command=close_window, bg="red", fg="white", font=("Helvetica", 14))
close_button.place(relx=0.97, rely=0.01, relwidth=0.03, relheight=0.03)
    
root.mainloop()